[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aioaneia/semantic-papers-filter/blob/main/notebooks/exploration.ipynb)

# Semantic Filter for Virology Papers

### **Clone** the repository https://github.com/aioaneia/semantic-papers-filter.git

In [8]:
! git clone https://github.com/aioaneia/semantic-papers-filter.git

Cloning into 'semantic-papers-filter'...
fatal: could not read Username for 'https://github.com': No such device or address


In [7]:
cd /content/semantic-papers-filter

[Errno 2] No such file or directory: '/content/semantic-papers-filter'
/content


### Import modules

In [ ]:
import sys
sys.path.append('/src/.')

import main.PaperFilteringApp as PaperFilteringApp

### Run the application.

In [ ]:
app = PaperFilteringApp(
  '../config.yaml'
)

app.run()

## Plot and save statistics

In [ ]:
pipeline.plot_statistics("../results")